In [1]:
import sacc
from sacc import Sacc, standard_types
import twopoint
from twopoint import Types

Load in the sacc data file from fits method

In [2]:
path = 'your_path'
filename = 'summary_statistics_real'
s = sacc.Sacc.load_fits(path+filename+'.sacc')

This table converts the type codes used in sacc to twopoint names

In [3]:
types = {
    standard_types.galaxy_shear_xi_plus: (Types.galaxy_shear_plus_real, Types.galaxy_shear_plus_real),
    standard_types.galaxy_shear_xi_minus: (Types.galaxy_shear_minus_real, Types.galaxy_shear_minus_real),
    standard_types.galaxy_shearDensity_xi_t: (Types.galaxy_position_real, Types.galaxy_shear_plus_real),
    standard_types.galaxy_density_xi: (Types.galaxy_position_real, Types.galaxy_position_real),
}

Use builder class from twopoint to build file interating overdata points.

In [4]:
builder = twopoint.SpectrumCovarianceBuilder()
for dt in s.get_data_types():
    data = s.get_data_points(dt)
    angbin = 0
    ang_array = [0]
    for datapoint in data:
        stype1 = types[datapoint.data_type][0]
        stype2 = types[datapoint.data_type][1]
        #In sacc we make a new tracer for each bin whereas in 
        # TwoPoint n(z) kernels are grouped into general types. 
        if 'lens_' in datapoint.tracers[0]: kernel1 = 'NZ_LENS'
        if 'lens_' in datapoint.tracers[1]: kernel2 = 'NZ_LENS'  
        if 'source_' in datapoint.tracers[0]: kernel1 = 'NZ_SOURCE'
        if 'source_' in datapoint.tracers[1]: kernel2 = 'NZ_SOURCE' 
        
        # get redshift bin number from the tracers information in sacc
        bin1 = int(datapoint.tracers[0][-1]) + 1
        bin2 = int(datapoint.tracers[1][-1]) + 1
        value = datapoint.value
        # for some reason some variables need to be called as dictionaries
        # instead of atributes
        ang = datapoint['theta']
        # Check whether to reset the angular bin to 1 or sum 1. 
        if ang < ang_array[-1]: 
            angbin = 1
        else:
            angbin +=1
        ang_array.append(ang)
        #print(kernel1,kernel2, bin1, bin2, ang, angbin)
        builder.add_data_point(kernel1,kernel2,stype1,stype2,bin1+1,bin2+1,ang,angbin,value)
        
# Define names of the spectra, might need tweaking      
names = {builder.types[0]:"wtheta", builder.types[1]:"gammat", builder.types[2]:"xim", builder.types[3]:"xip"}
builder.set_names(names)

# Load and add covariance
covmat = s.covariance.covmat

# Bulding spectra and conv matrix objects
spectra, covmat_info = builder.generate(covmat,"arcmin")

Create the number density objects not created using the builder class

In [5]:
nzs_l = []
nzs_s = []
for tracername in s.tracers:
    tracer = s.get_tracer(tracername)
    if 'lens' in tracername:
        z_l = tracer.z
        nzs_l.append(tracer.nz)
    if 'source' in tracername:
        z_s = tracer.z
        nzs_s.append(tracer.nz)

def zlow(z):
    return z - (z[1] - z[0])/2.

def zhigh(z):
    return z + (z[1] - z[0])/2.

nz_lens = twopoint.NumberDensity('NZ_LENS', zlow(z_l), z_l, zhigh(z_l), nzs_l)
nz_source = twopoint.NumberDensity('NZ_SOURCE', zlow(z_s), z_s, zhigh(z_s), nzs_s)

Create then TwoPointFile Object to save to fits file

In [7]:
twopointfile = twopoint.TwoPointFile(spectra, [nz_lens, nz_source], windows=None, covmat_info=covmat_info)
twopointfile.to_fits(path + filename + '_from_sacc.fits', overwrite=True)